In [15]:
import xml.etree.ElementTree as ET
from xml.dom import minidom

def extract_non_nested_fields_to_dict(xsd_file):
    # Parse the XML schema file
    tree = ET.parse(xsd_file)
    root = tree.getroot()

    # Initialize an empty dictionary to store required field names with empty string values
    required_fields_dict = {}

    # Namespace used in XSD files
    ns = {'xs': 'http://www.w3.org/2001/XMLSchema'}

    # Extract elements that do not have any <xs:element> children
    for element in root.findall(".//xs:element", ns):
        field_name = element.get("name")
        min_occurs = element.get("minOccurs")

        # Check if the element has children that are <xs:element>
        has_element_children = any(child.tag.endswith("element") for child in element.findall(".//*", ns))

        # Only add elements that do not have <xs:element> children
        if field_name and (min_occurs is None or min_occurs != "0") and not has_element_children:
            required_fields_dict[field_name] = ""

    return required_fields_dict


def create_xml_from_dict(field_dict, output_xml_file):
    # Create root element for the XML file
    deklaracja = ET.Element("Deklaracja")

    # Create Naglowek section
    naglowek = ET.SubElement(deklaracja, "Naglowek")
    ET.SubElement(naglowek, "KodFormularza", {
        "kodSystemowy": "PCC-3 (6)",
        "kodPodatku": "PCC",
        "rodzajZobowiazania": "Z",
        "wersjaSchemy": "1-0E"
    }).text = field_dict.get("KodFormularza", "")
    ET.SubElement(naglowek, "WariantFormularza").text = ""
    ET.SubElement(naglowek, "CelZlozenia", {"poz": "P_6"}).text = field_dict.get("CelZlozenia", "")
    ET.SubElement(naglowek, "Data", {"poz": "P_4"}).text = field_dict.get("Data", "")
    ET.SubElement(naglowek, "KodUrzedu").text = field_dict.get("KodUrzedu", "")

    # Create Podmiot1 section
    podmiot1 = ET.SubElement(deklaracja, "Podmiot1", {"rola": "Podatnik"})
    osoba_fizyczna = ET.SubElement(podmiot1, "OsobaFizyczna")
    ET.SubElement(osoba_fizyczna, "NIP").text = field_dict.get("NIP", "")
    ET.SubElement(osoba_fizyczna, "ImiePierwsze").text = field_dict.get("ImiePierwsze", "")
    ET.SubElement(osoba_fizyczna, "Nazwisko").text = field_dict.get("Nazwisko", "")
    ET.SubElement(osoba_fizyczna, "DataUrodzenia").text = field_dict.get("DataUrodzenia", "")
    ET.SubElement(osoba_fizyczna, "ImieOjca").text = field_dict.get("ImieOjca", "")
    ET.SubElement(osoba_fizyczna, "ImieMatki").text = field_dict.get("ImieMatki", "")

    # Create AdresZamieszkaniaSiedziby section
    adres_zamieszkania = ET.SubElement(podmiot1, "AdresZamieszkaniaSiedziby", {"rodzajAdresu": "RAD"})
    adres_pol = ET.SubElement(adres_zamieszkania, "AdresPol")
    ET.SubElement(adres_pol, "KodKraju").text = field_dict.get("KodKraju", "PL")
    ET.SubElement(adres_pol, "Wojewodztwo").text = field_dict.get("Wojewodztwo", "")
    ET.SubElement(adres_pol, "Powiat").text = field_dict.get("Powiat", "")
    ET.SubElement(adres_pol, "Gmina").text = field_dict.get("Gmina", "")
    ET.SubElement(adres_pol, "Ulica").text = field_dict.get("Ulica", "")
    ET.SubElement(adres_pol, "NrDomu").text = field_dict.get("NrDomu", "")
    ET.SubElement(adres_pol, "NrLokalu").text = field_dict.get("NrLokalu", "")
    ET.SubElement(adres_pol, "Miejscowosc").text = field_dict.get("Miejscowosc", "")
    ET.SubElement(adres_pol, "KodPocztowy").text = field_dict.get("KodPocztowy", "")

    # Create PozycjeSzczegolowe section
    pozycje_szczegolowe = ET.SubElement(deklaracja, "PozycjeSzczegolowe")
    for field, value in field_dict.items():
        if field not in ["NIP", "ImiePierwsze", "Nazwisko", "DataUrodzenia", "ImieOjca", "ImieMatki", "KodKraju", "Wojewodztwo", "Powiat", "Gmina", "Ulica", "NrDomu", "NrLokalu", "Miejscowosc", "KodPocztowy"]:
            ET.SubElement(pozycje_szczegolowe, field).text = value

    # Create Pouczenia section
    ET.SubElement(deklaracja, "Pouczenia").text = field_dict.get("Pouczenia", "")

    # Convert the tree to a string and prettify using minidom
    rough_string = ET.tostring(deklaracja, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    pretty_xml_as_string = reparsed.toprettyxml(indent="  ")

    # Write the prettified XML to the output file
    with open(output_xml_file, 'w', encoding='utf-8') as f:
        f.write(pretty_xml_as_string)

    print("XML file created with populated required fields at:", output_xml_file)

# Example usage
xsd_file = 'schema.xsd'  # Path to the XSD file
output_xml_file = 'output.xml'  # Output XML file path

# Extract fields into a dictionary with empty values
field_dict = extract_non_nested_fields_to_dict(xsd_file)

# Populate the dictionary with values
field_dict["KodFormularza"] = "PCC-3"
field_dict["WariantFormularza"] = "6"
field_dict["CelZlozenia"] = "1"
field_dict["P_20"] = "1"

# print number of fields with empty string values in the dictionary
print(len(field_dict))
print(field_dict)

# Create XML file using the dictionary values
create_xml_from_dict(field_dict, output_xml_file)


63
{'KodFormularza': 'PCC-3', 'WariantFormularza': '6', 'CelZlozenia': '1', 'Data': '', 'KodUrzedu': '', 'NIP': '', 'PESEL': '', 'ImiePierwsze': '', 'Nazwisko': '', 'DataUrodzenia': '', 'PelnaNazwa': '', 'SkroconaNazwa': '', 'AdresPol': '', 'AdresZagr': '', 'KodKraju': '', 'Wojewodztwo': '', 'Powiat': '', 'Gmina': '', 'NrDomu': '', 'Miejscowosc': '', 'KodPocztowy': '', 'Naglowek': '', 'OsobaNiefizyczna': '', 'AdresZamieszkaniaSiedziby': '', 'P_7': '', 'P_20': '1', 'P_23': '', 'P_24': '', 'P_25': '', 'P_26': '', 'P_27': '', 'P_28': '', 'P_29': '', 'P_30': '', 'P_31': '', 'P_32': '', 'P_33': '', 'P_34': '', 'P_35': '', 'P_36': '', 'P_37': '', 'P_38': '', 'P_39': '', 'P_40': '', 'P_41': '', 'P_42': '', 'P_43A': '', 'P_43': '', 'P_44': '', 'P_45': '', 'P_47': '', 'P_48': '', 'P_49': '', 'P_51': '', 'P_52': '', 'P_53': '', 'P_54': '', 'P_55': '', 'P_56': '', 'P_58': '', 'P_60': '', 'P_61': '', 'Pouczenia': ''}
XML file created with populated required fields at: output.xml
